In [2]:
import pandas as pd 
import numpy as np 
import plotly.express as px
import re

In [2]:
# https://www.cbs.nl/nl-nl/maatwerk/2024/35/kerncijfers-wijken-en-buurten-2024
df = pd.read_excel('./kwb-2024.xlsx')
df.shape

FileNotFoundError: [Errno 2] No such file or directory: './kwb-2024.xlsx'

In [ ]:
df.head()

,gwb_code_10,gwb_code_8,regio,gm_naam,recs,gwb_code,ind_wbi,a_inw,a_man,a_vrouw,...,g_afs_kv,g_afs_sc,g_3km_sc,a_opp_ha,a_lan_ha,a_wat_ha,pst_mvp,pst_dekp,ste_mvs,ste_oad
0,NL00,0000,Nederland,Nederland,Land,NL00,.,17942942,8920768,9022174,...,.,.,.,4154337,3364723,789614,.,.,2,2070
1,GM0014,0014,Groningen,Groningen,Gemeente,GM0014,1,243768,120912,122856,...,.,.,.,19796,18553,1244,.,.,1,3472
2,WK001400,001400,Centrum,Groningen,Wijk,WK001400,1,24920,12845,12070,...,.,.,.,241,228,13,.,.,1,6766
3,BU00140000,00140000,Binnenstad-Noord,Groningen,Buurt,BU00140000,1,4745,2470,2270,...,.,.,.,39,37,2,9712,1,1,7001
4,BU00140001,00140001,Binnenstad-Zuid,Groningen,Buurt,BU00140001,1,6975,3640,3340,...,.,.,.,59,55,4,9711,1,1,6698


In [ ]:
df = df[df['recs'] != "Land"]

In [ ]:
def plot_best_chances_for_singles(df, col, age_classes, min_population=10000, min_percentage=0.1):
    """
    Plots the regions or municipalities where single men have the best chances 
    based on the proportion of single women and a minimum population threshold.

    Args:
        df (pd.DataFrame): Input DataFrame with columns for demographics.
        col (str): The column to group by, e.g., 'gm_naam' or 'regio'.
        age_classes (list): List of age class columns to analyze, e.g., ['a_25_44', 'a_15_24'].
        min_population (int): Minimum population size to include in the analysis.
        min_percentage (float): Minimum proportion of single women to include (e.g., 0.1 for 10%).
    """
    # Ensure valid rows
    df = df.dropna(subset=["a_ongeh", "a_vrouw", "a_inw"] + age_classes)
    df = df[df["a_inw"] > 0]  # Exclude rows with zero population to avoid division errors

    for age_class in age_classes:
        if age_class not in df.columns:
            print(f"Age class '{age_class}' is not in the DataFrame. Skipping...")
            continue
        
        # Step 1: Calculate unmarried women proportion
        df["unmarried_women"] = df["a_ongeh"] * (df["a_vrouw"] / df["a_inw"])
        
        # Step 2: Adjust for the age group
        df["unmarried_women_age_group"] = df["unmarried_women"] * (df[age_class] / df["a_inw"])
        
        # Step 3: Calculate proportion of unmarried women relative to total population
        df["unmarried_women_proportion"] = df["unmarried_women_age_group"] / df["a_inw"]
        
        # Step 4: Filter by minimum population size and unmarried women proportion
        df_filtered = df[(df["a_inw"] >= min_population) & 
                         (df["unmarried_women_proportion"] >= min_percentage)]
        
        # Step 5: Get the top 10 regions based on unmarried women proportion
        top_10 = df_filtered.nlargest(10, "unmarried_women_proportion")
        
        # Step 6: Plot the data
        fig = px.bar(
            top_10,
            x=col,
            y="unmarried_women_proportion",
            title=f"Top 10 {col} Where Single Men Have the Best Chances ({age_class})",
            labels={col: col.capitalize(), "unmarried_women_proportion": "Proportion of Single Women"},
            text="unmarried_women_proportion",
        )
        fig.update_traces(texttemplate="%{text:.2%}", textposition="outside")
        fig.update_layout(
            xaxis_title=col.capitalize(),
            yaxis_title="Proportion of Single Women",
            uniformtext_minsize=8,
            uniformtext_mode="hide"
        )
        fig.show()

# Example usage
plot_best_chances_for_singles(df, col="regio", age_classes=["a_25_44", "a_15_24"], min_population=10000, min_percentage=0.15)

In [ ]:
df.columns

Index(['gwb_code_10', 'gwb_code_8', 'regio', 'gm_naam', 'recs', 'gwb_code',
       'ind_wbi', 'a_inw', 'a_man', 'a_vrouw',
       ...
       'g_afs_kv', 'g_afs_sc', 'g_3km_sc', 'a_opp_ha', 'a_lan_ha', 'a_wat_ha',
       'pst_mvp', 'pst_dekp', 'ste_mvs', 'ste_oad'],
      dtype='object', length=131)

In [ ]:
# rotate proxy
# robot.xtx
# make sure it has more delay


In [3]:
data = pd.read_csv("./data/data.csv")
data.shape

(39111, 9)

In [4]:
data.columns

Index(['name', 'adres', 'website', 'telefoon', 'reviews_count',
       'reviews_average', 'latitude', 'longitude', ' search_keyword'],
      dtype='object')

In [5]:
data = data.sort_values(by='reviews_count', ascending=False)
data = data.drop_duplicates(subset=['name', 'website'], keep='first')

# Drop rows where 'website' is NaN or an empty string
data = data[data['website'].notna() & (data['website'] != "")]
data.shape

(11616, 9)

In [ ]:
# use data seen to not read df every loop to make it faster
# Scrape only data that has websites for faster handle 
# update script to be callable by class to run paralel without copy .py
# Modify the code so it can run multi browser and boost speed.
# Append keyword used into the csv for storage so we know where it came from
# maybe append takes much ram on csv

In [ ]:
# XML converter
# Git page readme 
# update warmup to single mailer